{{ badge }}

 ```
http://bit.ly/VITMAV45-task1
```


# Copyright

<PRE>
Jelen Jupyter notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.
A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A notebook bármely részének újra felhasználása, publikálása csak a szerzők írásos beleegyezése esetén megegengedett.

2019 (c) Gyires-Tóth Bálint (toth.b kukac tmit pont bme pont hu)
</PRE>

# Regresszió mély tanulással

Ebben a notebookban a gépi tanulás egyik "Hello world"-jét fogjuk megismeri. Ehhez a Californa Housing Prices adatbázist fogjuk felhasználni.

Az adatbázisról egy rövid leírást itt találunk:
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

Részletesebb leírást, elemzést itt:
https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html

Következő lépésként importáljuk a szükséges modulokat és állítsunk be egy random seed-et:

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import copy
import pandas as pd
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler


np.random.seed(123)

Ezután töltsük le és töltsük be az adatokat és vizsgáljuk meg hogy milyen oszlopokat (bemenő változók és cél/becsült változó) tartalmaz.

In [3]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)

In [4]:
california_housing.frame

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


Válasszuk szét a model be- és kimenetét jelentő változókat,  train-validation-test adathalmazokat és standardizáljuk a bemeneteket

In [5]:
from sklearn import preprocessing

dataset = california_housing.frame.values
test_split = 0.1
valid_split = 0.1

# Be és kimenetek szétválasztása
X = dataset[:,:-1]
Y = dataset[:,-1]

display(X)
# Adatok szétválasztása train-validation-test partíciókrart


X_valid_start_idx =int(len(X)*(1-valid_split-test_split))
X_test_start_idx = int((1-test_split)*len(X))

X_test = X[X_test_start_idx:]
Y_test = Y[X_test_start_idx:]
X_valid = X[X_valid_start_idx:X_test_start_idx]
Y_valid = Y[X_valid_start_idx:X_test_start_idx]
X = X[:X_valid_start_idx]
Y = Y[:X_valid_start_idx]

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

Hozzuk létre a hálót és tanítsuk be!

**Biztosítsd a legjobb modell kimentését a `weights.keras` fájlba**

<details>
  <summary>Tippek háló és tanítás összeállításhoz. Ha pro vagy próbáld enélkül.</summary>

- A háló 2 egyszerű neuron rétegből álljon
    1. réteg
        - bemeneteinek száma a bemenő változók száma
        - 200 neuront tartalmazzon
        - Szigmoid aktivációs függvény
    2. réteg
        - ez adja a kimenetet, tehát a célváltozók számával egyezen meg a neuronok száma (units)
        - aktivációs függvénynek gondold át mit érdemes ehhez a feladathoz használni(`linear`, `sigmoid`, `tanh`, ...)
- Optimalicációhoz SGD algoritmust használj, $10^{-3}$ tanulási rátával,  $10^{-6}$ L2 regularizáció együtthatóval (weight decay), valamint a következő momentum beállítással: `momentum=0.9, nesterov=True`
- Hibafüggvény: négyzetes hiba
- Batch méret: 16
- Használj callbackeket, amelyek biztosítják, hogy:
    - A tanítás álljon le, ha a 40 epochon át nem javul a validációs hiba
    - A legjobb modelt elmentjük a `weights.keras` fájlba.

> Ha sikerült a tanítás kódját összerakni elindítani, akkor térj ide vissza és módosítsd, a két réteg közé adj hozzá egy Dropout(0.5) réteget!

  ```
</details>


In [6]:
from os import access
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.random import set_seed

set_seed(2)


model = Sequential()
model.add(layers.Dense(200, activation='sigmoid', input_dim=X.shape[1]))
model.add(  layers.Dense(1,activation='linear'))

sgd = SGD(learning_rate=.001,momentum=.9,nesterov=True,weight_decay=.000001)
model.compile(loss='mean_squared_error',
              optimizer= sgd,
              metrics=['accuracy'])


from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer=ModelCheckpoint(filepath='weights.keras', save_best_only=True, verbose=1)


from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(patience=40, verbose=1)


history= model.fit(X,Y,
                   batch_size=16,
                   verbose=2,
                   validation_data=(X_valid,Y_valid),
                   epochs=1000,
                   callbacks=[checkpointer,early_stopping],
                   shuffle=True)

/home/scsng/BME/DeepLearning/kisHF/HF1/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1727374648.429666   19012 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-26 20:17:28.641385: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required librar

Epoch 1/1000

Epoch 1: val_loss improved from inf to 0.80900, saving model to weights.keras
1032/1032 - 4s - 3ms/step - accuracy: 0.0031 - loss: 0.6968 - val_accuracy: 0.0000e+00 - val_loss: 0.8090
Epoch 2/1000

Epoch 2: val_loss improved from 0.80900 to 0.68132, saving model to weights.keras
1032/1032 - 2s - 2ms/step - accuracy: 0.0032 - loss: 0.5456 - val_accuracy: 0.0000e+00 - val_loss: 0.6813
Epoch 3/1000

Epoch 3: val_loss improved from 0.68132 to 0.62885, saving model to weights.keras
1032/1032 - 2s - 2ms/step - accuracy: 0.0032 - loss: 0.5155 - val_accuracy: 0.0000e+00 - val_loss: 0.6288
Epoch 4/1000

Epoch 4: val_loss improved from 0.62885 to 0.60783, saving model to weights.keras
1032/1032 - 2s - 2ms/step - accuracy: 0.0033 - loss: 0.5034 - val_accuracy: 0.0000e+00 - val_loss: 0.6078
Epoch 5/1000

Epoch 5: val_loss improved from 0.60783 to 0.59697, saving model to weights.keras
1032/1032 - 2s - 2ms/step - accuracy: 0.0032 - loss: 0.4973 - val_accuracy: 0.0000e+00 - val_loss: 0

KeyboardInterrupt: 

A tanítás végeztével visszatöltjük a legjobb modellt, és kiértékeljük ennek teljesítményét a teszt adatokon. Mit jelenthet ez az érték?

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('weights.keras')


################################################################################
# KIÉRTÉKELÉS
################################################################################

from sklearn.metrics import mean_squared_error
import math

preds = model.predict(X_test)
from sklearn.metrics import mean_squared_error
test_err = mean_squared_error(Y_test,preds)

print("\nTeszt hiba: %f" % (test_err))
print(f"Ez az jelenti, hogy sqrt({test_err:0.3f}))={math.sqrt(test_err):0.3f}-at hibáz átlagosan a modell a teszt adatokon.")
print(f"Ez annyit jelent, hogy átlagosan {math.sqrt(test_err)*100000:0.0f}$-t téved a modellünk.")

Az eredményeket vizsgáljuk meg grafikusan is:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
fig = sns.regplot(x=Y_test, y=preds.reshape(-1));
# fig.set(xlim=(10,30),ylim=(10,30))
plt.xlabel("Ground truth [$ 100 000]")
plt.ylabel("Predictions [$ 100 000]")
plt.show()

Mit jelent ez az ábra? Mit jelentene egy vízszintes egyenes? És egy függőleges egyenes?